In [1]:
#package imports
import json
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/4b452d80-e019-43df-
[nltk_data]     abf4-e400fc24dfd9/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#security related keywords
security_words = ['buffer', 'corrupt', 'boundary', 'valid', 'invalid','bypass','neutralize','restrict','pathname','verify','query','request','authorization', 'authentication','command','permission','password','key','deserialize','encrypt','decrypt','credentials','authorize','authenticate','restricted','neutralized','corruption','permissions','verifies','verified','validated','invalidated','authorized','unauthorized','aunthenticate','passkey']
tokenized_security_words = [word_tokenize(word) for word in security_words]

In [3]:
memory_fxns = ['malloc','calloc','memcpy','free','realloc','alloc','memchr','memcmp','memmove','memset']

In [ ]:
# json_file = 'CVE-1999-0199.json'
# with open(json_file, 'r') as file:
#     data = json.load(file)

In [ ]:
# ignore this block
file_changes = data['file_changes']
for i in file_changes:
    for j in file_changes[i]:
        if j != 'used_context_lines':
            k = 0
            while k < len(file_changes[i][j]):
                line = file_changes[i][j][k]
                #print(line)
                if( line != "" and line[0] == '-'):
                    print(line)
                k += 1
            # print(file_changes[i][j][0])
            # k = 0
            # while k < len(file_changes[i][j]):
            #     if(file_changes[i][j][k] == '-'):
            #         print(file_changes[i][j][k])
            #         #print(len(word_tokenize(file_changes[i][j][k])))
            #     k += 1

In [ ]:
# ignore this block
# test = file_changes['elf/dl-load.c']['malicious_lines']
# print(test[0][0])

In [5]:
def update_csv(json_file, csv_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        
        file_changes = data['file_changes']
        mem_words_present = []
        tokens_added = 0
        tokens_removed = 0

        for i in file_changes:
            for j in file_changes[i]:
                if j != 'used_context_lines':
                    #tokens added or removed
                    k = 0
                    while k < len(file_changes[i][j]):
                        line = file_changes[i][j][k]
                        if( line != "" and line[0] == '-'):
                            tokens_removed = tokens_removed + len(word_tokenize(line))
                        if (line != "" and line[0] == '+'):
                            tokens_added = tokens_added + len(word_tokenize(line))
                    k += 1
        #will fix this later
        for i in file_changes:
            for j in file_changes[i]:
                if j != 'used_context_lines':
                    mem_words_present = mem_words_present + [word for word in memory_fxns if word in file_changes[i][j]]
            
        #metadata
        metadata = data['commit_metadata']
        #basic counts
        author_length = len(metadata['author'])
        email_length = len(metadata['author_email'])
        msg_length = len(metadata['message'])
        #tokens
        sec_words_present = [word for word in security_words if word in metadata['message'].lower()]
        token_msg = word_tokenize(metadata['message'])
        tokens = len(token_msg)
        df.loc[len(df.index)] = [author_length, email_length, msg_length, sec_words_present, mem_words_present, tokens_added, tokens_removed]
    #display(df)          
    df.to_csv(csv_file, index = False)
#test run
#update_csv('jsonfolder/CVE-1999-0199.json', 'character_counts.csv')

In [ ]:
json_folder = 'commit_metadata'
csv_file = 'character_counts.csv'
df = pd.DataFrame(columns = ['Length of author name', 'Length of author email', 'Length of commit message', 'Security related keywords in message', 'Memory related functions in code changes', 'Tokens added in commit', 'Tokens removed in commit'])
for file in os.listdir(json_folder):
    if file.endswith('.json'):
        json_file = os.path.join(json_folder, file)
        update_csv(json_file, csv_file)

In [4]:
# ignore this block
# current_directory = 'commit_metadata'

# files = os.listdir(current_directory)

# for file in files:
#     if file.endswith('.json'):
#         file_path = os.path.join(current_directory, file)
#         os.remove(file_path)